In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.datasets import boston_housing
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import Sequential

In [3]:
(train_data , train_result),(test_data,test_results) = boston_housing.load_data()

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step


In [8]:
train_data.shape

(404, 13)

We need to normalize the data to allow the training to be more easy .
We never use the test data , not even for data normalisation . Always use the train data !

In [10]:
mean = train_data.mean(axis=0)
train_data -= mean

stdd = train_data.std(axis=0)
train_data /= stdd

test_data -= mean
test_data /= stdd


Now we can build the NN

Why do we use the ReLu activation function ?  

For the vanishing gradient issue !

In fact , if we use for example the sigmoid function . We know that we use the chain rule do differentiate the Cost function ,  
and then σ′(x)=σ(x)(1−σ(x))≤0.25 .  

Therefore after a lot of layers the gradient converges to zero and the lowest layers are not learning anything !

But if we use the ReLu function : f(x) = max(0,x) . We can assure that the gradient is higher than one  !

In [11]:
def build_model():
    model = Sequential([
        Dense(64,activation="relu"),
        Dense(64,activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer = "rmsprop", loss='mse' , metrics=['mae'])
    return model

To evaluate our model while we keep adjusting its parameters (such as the number of
epochs used for training) wre use the K-Fold validation.  

It consists of splitting the available data into K partitions (typically K = 4 or 5), instanti-
ating K identical models, and training each one on K – 1 partitions while evaluating
on the remaining partition. The validation score for the model used is then the aver-
age of the K validation scores obtained

In [15]:
k = 4
all_score = []
samples_per_fold= len(train_data) // 4
for i in range(k):
    print(f'Processing fold number : {i}')
    val_data = train_data[i * samples_per_fold : (i+1)*samples_per_fold]
    val_target = test_data[i * samples_per_fold : (i+1)*samples_per_fold]
    
    partial_data_train = np.concatenate(
        [
        train_data[: i*samples_per_fold],
        train_data[(i+1)*samples_per_fold:]]
    )
    partial_result_train = np.concatenate(
        [
        test_data[: i*samples_per_fold],
        test_data[(i+1)*samples_per_fold:]]
    )
    
    model = build_model()
    model.fit(partial_data_train,partial_result_train, epochs= 100 , batch_size= 16)
    val_mse, val_mae = model.evaluate(val_data, val_target, verbose=0)
    all_score.append(val_mae)

Processing fold number : 0


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 303
'y' sizes: 1
